In [ ]:
import numpy as np
from keras import ops
import matplotlib.pyplot as plt

import zea
from ulsa.ops import iq2doppler,tissue_doppler_strain_rate

zea.init_device()

In [ ]:
from matplotlib import animation
from IPython.display import HTML

def animate_images(images, scan, interval=100, cmap="gray"):
    """Helper function to animate a list of images."""
    fig, ax = plt.subplots()
    xlims_mm = [v * 1e3 for v in scan.xlims]
    zlims_mm = [v * 1e3 for v in scan.zlims]
    im = ax.imshow(
        np.array(images[0]),
        animated=True,
        cmap=cmap,
        extent=[xlims_mm[0], xlims_mm[1], zlims_mm[1], zlims_mm[0]],
    )
    ax.set_xlabel("X (mm)")
    ax.set_ylabel("Z (mm)")

    def update(frame):
        im.set_array(np.array(images[frame]))
        return [im]

    ani = animation.FuncAnimation(fig, update, frames=len(images), blit=True, interval=interval)
    plt.close(fig)
    return HTML(ani.to_jshtml())

In [ ]:
path = "/mnt/USBMD_datasets/2023_USBMD_rotating_disc_L115v/L115V_1radsec.hdf5"
with zea.File(path) as file:
    data = file.load_data("raw_data")
    scan = file.scan()
    probe = file.probe()

In [ ]:
pipeline = zea.Pipeline.from_default(with_batch_dim=True)
params = pipeline.prepare_parameters(probe, scan)
output = pipeline(data=data, **params)
bmode = output["data"]

In [ ]:
animate_images(bmode, scan)

In [ ]:
pipeline = zea.Pipeline(
    [
        zea.ops.Demodulate(),
        zea.ops.PatchedGrid(
            [zea.ops.TOFCorrection(), zea.ops.DelayAndSum()], num_patches=10
        ),
        zea.ops.ChannelsToComplex(),
    ],
    jit_options="pipeline",
)

In [ ]:
transmits = "all"
scan.set_transmits(transmits)
params = pipeline.prepare_parameters(probe, scan)
output = pipeline(data=data[:, scan.selected_transmits], **params)
data4doppler = output["data"]
data4doppler = ops.moveaxis(data4doppler, 0, -1)

In [ ]:
pulse_repetition_frequency = 1 / sum(scan.time_to_next_transmit[0])
d = iq2doppler(data4doppler[..., :], probe.center_frequency, pulse_repetition_frequency, scan.sound_speed, hamming_size=10)
plt.imshow(d, cmap="bwr")
plt.colorbar()

In [ ]:
spacing = (scan.zlims[1] - scan.zlims[0]) / (scan.n_z - 1)
strain_rate = tissue_doppler_strain_rate(ops.convert_to_numpy(d), axis=0, spacing=spacing, method="central")
plt.imshow(strain_rate, cmap="seismic")
plt.colorbar()